In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!yes | unzip /content/gdrive/MyDrive/151B_data/new_train.zip > /dev/null
!yes | unzip /content/gdrive/MyDrive/151B_data/new_val_in.zip > /dev/null

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os, os.path 
import numpy 
import pickle
from glob import glob


"""Change to the data folder"""
new_path = "data/new_train"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [7]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)

In [8]:
batch_sz = 100

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """

    inp = [[scene['p_in'], scene['v_in']] for scene in batch]
    out = [[scene['p_out'], scene['v_out']] for scene in batch]
    mask = [scene['car_mask'] for scene in batch]

    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    mask = torch.LongTensor(mask)
    return [inp, out, mask]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [ ]:
'''
max_p_x = 0
min_p_x = 5000
max_p_y = 0
min_p_y = 5000
max_v_x = 0
min_v_x = 100
max_v_y = 0
min_v_y = 100

for i_batch, sample_batch in enumerate(val_loader):
    inp, out, mask = sample_batch
    for i in inp:
      if i[0][:,0].max()> max_p_x:
        max_p_x = i[0][:,0].max()
      if i[0][:,0].min() < min_p_x:
        min_p_x = i[0][:,0].min()
      if i[0][:,1].max()> max_p_y:
        max_p_y = i[0][:,1].max()
      if i[0][:,1].min() < min_p_y:
        min_p_y = i[0][:,1].min()

      if i[1][:,0].max()> max_v_x:
        max_v_x = i[1][:,0].max()
      if i[1][:,0].min()< min_v_x:
        min_v_x = i[1][:,0].min()
      if i[1][:,1].max()> max_v_y:
        max_v_y = i[1][:,1].max()
      if i[1][:,1].min()< min_v_y:
        min_v_y = i[1][:,1].min()

    for o in out:
      if o[0][:,0].max()> max_p_x:
        max_p_x = o[0][:,0].max()
      if o[0][:,0].min() < min_p_x:
        min_p_x = o[0][:,0].min()
      if o[0][:,1].max()> max_p_y:
        max_p_y = o[0][:,1].max()
      if o[0][:,1].min() < min_p_y:
        min_p_y = o[0][:,1].min()

      if o[1][:,0].max()> max_v_x:
        max_v_x = o[1][:,0].max()
      if o[1][:,0].min()< min_v_x:
        min_v_x = o[1][:,0].min()
      if o[1][:,1].max()> max_v_y:
        max_v_y = o[1][:,1].max()
      if o[1][:,1].min()< min_v_y:
        min_v_y = o[1][:,1].min()

    if(i_batch%100==0):
      print(i_batch)

print(max_p_x, min_p_x, max_p_y, min_p_y) # [4800,-50,4800,-50]
print(max_v_x, min_v_x, max_v_y, min_v_y) # [200,-200,200,-200]
'''

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Trajectory(nn.Module):

    def __init__(self):
        super(Trajectory, self).__init__()

        self.p_in = nn.Linear(2,32) # N, 19, 2
        self.v_in = nn.Linear(2,32) # N, 19, 2
        
        self.encoder = nn.LSTM(64, 64, 1) # input 19, N, 4 output 19, N, 64

        self.decoder_p = nn.LSTM(64, 128,1) # input 30, N, 64 output 30, N, 128
        self.decoder_v = nn.LSTM(64, 128,1) # input 30, N, 64 output 30, N, 128

        self.p_out = nn.Linear(128,2)
        self.v_out = nn.Linear(128,2)

    def forward(self, p, v, holder):
        batch = p.shape[0]
        x_p = self.p_in(p)
        x_v = self.v_in(v)

        x = torch.cat((x_p,x_v),dim=2)
        x = x.view(19,batch,64)
        x,_ = self.encoder(x)

        x = torch.cat((x,holder),dim=0)

        x_p,_ = self.decoder_p(x)
        x_v,_ = self.decoder_v(x)

        x_p = x_p.view(batch,30,128)
        x_v = x_v.view(batch,30,128)
        x_p = self.p_out(x_p)
        x_v = self.v_out(x_v)

        return x_p, x_v

In [ ]:
'''
import numpy as np
model = Trajectory().double()
print(model)

p = np.zeros((4,19,2))
v = np.zeros((4,19,2))
p_torch = torch.tensor(p)
v_torch = torch.tensor(v)

output = model(p_torch, v_torch)
print(len(output))
print(output[0].shape)
'''

In [11]:
from tqdm import tqdm

In [16]:
model = Trajectory().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epoch = 2

for i in range(epoch):
  model.train()
  epoch_loss = 0
  for i_batch, sample_batch in tqdm(enumerate(val_loader), total=len(val_loader), leave=False):
    inp, out, mask = sample_batch
    batch = inp.shape[0]

    holder = torch.zeros((11,batch*60,64)).to(device)

    p_in = (inp[:,0].reshape(batch*60,19,2).to(device)-2400.)/2400.
    v_in = inp[:,1].reshape(batch*60,19,2).to(device)/200.
    p_out = (out[:,0].reshape(batch*60,30,2).to(device)-2400.)/2400.
    v_out = out[:,1].reshape(batch*60,30,2).to(device)/200.

    pred = model(p_in, v_in, holder)


    loss = 0
    p_criteria = nn.MSELoss()
    p_loss = torch.sqrt(p_criteria(pred[0], p_out))

    v_criteria = nn.MSELoss()
    v_loss = torch.sqrt(v_criteria(pred[1], v_out))

    if (i_batch%200==0 and i_batch!=0):
      print(pred[0][0]*2400.+2400., p_out[0]*2400.+2400.)
      print(pred[1][0]*200., v_out[0]*200.)
      print(i_batch, p_loss, v_loss)
    loss = p_loss + v_loss
    epoch_loss += loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(i, epoch_loss)


  0%|          | 0/2060 [00:00<?, ?it/s]<ipython-input-8-db1da268cb2b>:10: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  inp = torch.LongTensor(inp)
<ipython-input-8-db1da268cb2b>:11: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  out = torch.LongTensor(out)
<ipython-input-8-db1da268cb2b>:12: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  mask = torch.LongTensor(mask)
 10%|▉         | 201/2060 [03:19<31:15,  1.01s/it]

tensor([[2184.3801, 2322.3022],
        [2184.3555, 2322.3254],
        [2184.4783, 2322.4482],
        [2184.4783, 2322.4482],
        [2184.4927, 2322.4944],
        [2184.4927, 2322.4944],
        [2184.4985, 2322.4990],
        [2184.4985, 2322.4990],
        [2184.5371, 2322.5342],
        [2184.5371, 2322.5342],
        [2184.5486, 2322.4871],
        [2184.5752, 2322.5266],
        [2184.4321, 2322.4001],
        [2184.4705, 2322.4419],
        [2184.5552, 2322.4246],
        [2184.5552, 2322.4246],
        [2184.4707, 2322.3838],
        [2184.4707, 2322.3838],
        [2184.4502, 2322.3635],
        [2183.1079, 2321.0581],
        [2183.1079, 2321.0581],
        [2182.9958, 2320.9780],
        [2183.1079, 2321.0581],
        [2183.1079, 2321.0581],
        [2183.1079, 2321.0581],
        [2183.1079, 2321.0581],
        [2182.9958, 2320.9780],
        [2183.1079, 2321.0581],
        [2183.2141, 2321.1523],
        [2182.9910, 2320.9143]], device='cuda:0', grad_fn=<AddBackward0>

 19%|█▉        | 401/2060 [06:43<27:59,  1.01s/it]

tensor([[1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.5914, 1795.6309],
        [1665.9496, 1795.9873],
        [1665.9496, 1795.9873],
        [1665.9496, 1795.9873],
        [1665.9496, 1795.9873],
        [1665.9496, 1795.9873],
        [1665.9496, 1795.9873],
        [1665.9496, 1795.9873],
        [1665.9496, 1795.9873],
        [1665.9496, 1795.9873],
        [1665.9496, 1795.9873],
        [1665.9496, 1795.9873]], device='cuda:0', grad_fn=<AddBackward0>

 29%|██▉       | 601/2060 [10:09<25:09,  1.03s/it]

...

 87%|████████▋ | 1801/2060 [31:40<04:26,  1.03s/it]

tensor([[331.8955, 491.2869],
        [333.1819, 492.7561],
        [331.7388, 491.1604],
        [332.2490, 491.6084],
        [331.5278, 490.9834],
        [331.8289, 491.4150],
        [331.7871, 491.3694],
        [331.3308, 490.6200],
        [331.6692, 491.1448],
        [332.1619, 491.5892],
        [331.2571, 490.4918],
        [331.5166, 490.9136],
        [331.4036, 490.7313],
        [330.1069, 489.3029],
        [331.2317, 490.5721],
        [331.2029, 490.5437],
        [330.7625, 490.0775],
        [331.3887, 490.7374],
        [330.6292, 489.9110],
        [341.1306, 500.9581],
        [344.9363, 504.4686],
        [339.6003, 499.6525],
        [339.0591, 499.1278],
        [339.2324, 499.3733],
        [337.8984, 498.3087],
        [339.9507, 500.0834],
        [339.6926, 499.7108],
        [349.1062, 507.7505],
        [339.0842, 499.1754],
        [338.6040, 498.8973]], device='cuda:0', grad_fn=<AddBackward0>) tensor([[ 740.9999, 2181.0000],
        [ 740.9999, 2180.0

 97%|█████████▋| 2001/2060 [35:04<00:59,  1.01s/it]

tensor([[301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.6147, 472.4102],
        [301.9487, 472.5869],
        [301.9487, 472.5869],
        [301.9487, 472.5869],
        [301.9487, 472.5869],
        [301.9487, 472.5869],
        [301.9487, 472.5869],
        [301.9487, 472.5869],
        [301.9487, 472.5869],
        [301.9487, 472.5869],
        [301.9487, 472.5869],
        [301.9487, 472.5869]], device='cuda:0', grad_fn=<AddBackward0>) tensor([[ 790., 1984.],
        [ 790., 1984.],
        [ 

1 tensor(688.6138, device='cuda:0', grad_fn=<AddBackward0>)
